# Travel Package Booking Portal

This notebook demonstrates how to use the travel package booking portal to search for and book travel packages that combine flights and hotels.

## 1. Setup and Configuration

First, let's install the required dependencies:

In [ ]:
!pip install -r requirements.txt

Now, let's set up our API credentials. You'll need to obtain API keys for:
- Amadeus (for flight and hotel data): https://developers.amadeus.com/
- Groq (for AI-powered recommendations): https://console.groq.com/

In [ ]:
import os
import sys
import logging

# Add parent directory to path to import modules
sys.path.append('..')

# Set up API credentials
os.environ['AMADEUS_API_KEY'] = "YOUR_AMADEUS_API_KEY"  # Replace with your actual API key
os.environ['AMADEUS_API_SECRET'] = "YOUR_AMADEUS_API_SECRET"  # Replace with your actual API secret
os.environ['GROQ_API_KEY'] = "YOUR_GROQ_API_KEY"  # Replace with your actual API key
os.environ['GROQ_MODEL'] = "llama3-8b-8192"  # You can change this to a different model if needed

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('notebook')

## 2. Initialize APIs and Agent

Now, let's initialize the APIs and agent for our travel booking portal:

In [ ]:
from utils import Config, TravelAgent, DataProcessor
from api import FlightAPI, HotelAPI, PackageAPI

# Load configuration
config = Config()
if not config.validate():
    logger.error("Invalid configuration. Please check your API credentials.")
    raise ValueError("Invalid configuration")

# Initialize APIs
amadeus_credentials = config.get_amadeus_credentials()
flight_api = FlightAPI(amadeus_credentials['api_key'], amadeus_credentials['api_secret'])
hotel_api = HotelAPI(amadeus_credentials['api_key'], amadeus_credentials['api_secret'])
package_api = PackageAPI(flight_api, hotel_api)

# Initialize agent
groq_credentials = config.get_groq_credentials()
travel_agent = TravelAgent(groq_credentials['api_key'], groq_credentials['model'])

# Initialize data processor
data_processor = DataProcessor()

print("APIs and agent initialized successfully!")

## 3. Natural Language Query Processing

Let's use the travel agent to process a natural language query and extract travel criteria:

In [ ]:
# Example natural language query
query = "I want to travel from New York to Paris for a week in July with my family (2 adults, 1 child)"

# Extract travel criteria
criteria = travel_agent.extract_travel_criteria(query)

# Display extracted criteria
import json
print(json.dumps(criteria, indent=2))

## 4. Search for Flights

Now, let's search for flights based on the extracted criteria:

In [ ]:
# Set up search parameters
origin = criteria.get('origin_location', 'NYC')  # Default to NYC if not extracted
destination = criteria.get('destinations', 'PAR')  # Default to Paris if not extracted
if isinstance(destination, list):
    destination = destination[0]  # Take the first destination if multiple are extracted

# Set dates (default to next month if not extracted)
from datetime import datetime, timedelta
default_departure = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
default_return = (datetime.now() + timedelta(days=37)).strftime('%Y-%m-%d')
departure_date = criteria.get('departure_date', default_departure)
return_date = criteria.get('return_date', default_return)

# Set passenger counts
adults = criteria.get('adults', 2)
children = criteria.get('children', 1)

# Set travel class
travel_class = criteria.get('travel_class', 'ECONOMY')

# Search for flights
print(f"Searching for flights from {origin} to {destination} on {departure_date} returning {return_date}...")
flight_offers = flight_api.search_flights(
    origin_location_code=origin,
    destination_location_code=destination,
    departure_date=departure_date,
    return_date=return_date,
    adults=adults,
    children=children,
    travel_class=travel_class,
    currency_code="USD",
    max_results=5
)

# Display flight offers
print(f"Found {len(flight_offers)} flight offers")
if flight_offers:
    # Parse flight data into a DataFrame
    flights_df = data_processor.parse_flight_data(flight_offers)
    flights_df.head()

## 5. Search for Hotels

Now, let's search for hotels at the destination:

In [ ]:
# Search for hotels
print(f"Searching for hotels in {destination} from {departure_date} to {return_date}...")
hotel_offers = hotel_api.search_hotels(
    city_code=destination,
    check_in_date=departure_date,
    check_out_date=return_date,
    adults=adults,
    radius=5,
    radius_unit="KM",
    currency="USD",
    max_results=5
)

# Display hotel offers
print(f"Found {len(hotel_offers)} hotel offers")
if hotel_offers:
    # Parse hotel data into a DataFrame
    hotels_df = data_processor.parse_hotel_data(hotel_offers)
    hotels_df.head()

## 6. Create Travel Packages

Now, let's combine flights and hotels to create travel packages:

In [ ]:
# Create travel packages
print("Creating travel packages...")
packages = package_api.create_single_destination_package(
    origin_location_code=origin,
    destination_location_code=destination,
    departure_date=departure_date,
    return_date=return_date,
    adults=adults,
    children=children,
    travel_class=travel_class,
    currency_code="USD",
    max_flight_results=5,
    max_hotel_results=5
)

# Display packages
print(f"Created {len(packages)} travel packages")
if packages:
    # Parse package data into a DataFrame
    packages_df = data_processor.parse_package_data(packages)
    packages_df.head()

## 7. Get AI-Powered Recommendations

Now, let's use the travel agent to generate personalized recommendations based on the available packages:

In [ ]:
# Generate recommendations
if packages:
    print("Generating recommendations...")
    recommendations = travel_agent.generate_travel_recommendations(packages[:3], criteria)
    print("\nRecommendations:")
    print(recommendations)

## 8. View Package Details

Let's view the details of a specific package:

In [ ]:
# View package details
if packages:
    package_id = 0  # View the first package
    print(f"Viewing details for package {package_id}...")
    package_details = travel_agent.format_package_details(packages[package_id], detailed=True)
    print("\nPackage Details:")
    print(package_details)

## 9. Ask Travel-Related Questions

Finally, let's ask the travel agent some questions:

In [ ]:
# Ask a question
question = "What's the best time to visit Paris?"
print(f"Question: {question}")

# Prepare context with current search results and criteria
context = {
    'current_criteria': criteria,
    'has_search_results': len(packages) > 0,
    'number_of_packages': len(packages) if packages else 0,
    'price_range': {
        'min': min([p['total_price'] for p in packages]) if packages else None,
        'max': max([p['total_price'] for p in packages]) if packages else None,
        'currency': packages[0]['currency'] if packages else None
    }
}

# Get answer from agent
answer = travel_agent.answer_travel_question(question, context)
print("\nAnswer:")
print(answer)

## 10. Launch Gradio Web Interface

If you want to use the web interface instead of the notebook, you can launch the Gradio app:

In [ ]:
from frontend.app import app as gradio_app

# Launch the app
gradio_app.launch()